import joblib
from flask import Flask, request, render_template
import spacy
import fitz  # PyMuPDF for PDFs
from docx import Document
import re 

app = Flask(__name__)

# Load NLP and ML models
nlp = spacy.load("en_core_web_sm")
model = joblib.load("resume_classifier.pkl")
vectorizer = joblib.load("vectorizer.pkl")

# Job field requirements
job_requirements = {
    "Data Science": {"skills": ["Python", "Machine Learning", "Statistics", "SQL"]},
    "Software Engineer": {"skills": ["Java", "OOP", "Algorithms", "Databases"]},
    "Full Stack Developer": {"skills": ["HTML", "CSS", "JavaScript", "React", "Node.js"]},
    "Data Analytics": {"skills": ["Excel", "SQL", "Data Visualization", "Power BI"]}
}

# Skill set for extraction
skill_list = {"Python", "Machine Learning", "SQL", "Statistics", "Java", "React", "Power BI", "OOP", "CSS", "JavaScript"}

# Extract text from resume
def extract_text_from(file_path):
    if file_path.endswith('.pdf'):
        with fitz.open(file_path) as doc:
            return " ".join([page.get_text() for page in doc])
    elif file_path.endswith('.docx'):
        doc = Document(file_path)
        return " ".join([para.text for para in doc.paragraphs])
    else:
        return "Unsupported File Type"

# Extract skills from resume

def extract_skills(text):
    matched_skills = []
    for skill in skill_list:
        pattern = r'\b' + re.escape(skill.lower()) + r'\b'
        if re.search(pattern, text.lower()):
            matched_skills.append(skill)
    return matched_skills


# Predict job category using ML model
def predict_job_category(text):
    text_vectorized = vectorizer.transform([text])
    return model.predict(text_vectorized)[0]

# Check suitability based on skills
def check_suitability(predicted_category, extracted_skills):
    required_skills = set(job_requirements.get(predicted_category, {}).get("skills", []))
    matched_skills = set(extracted_skills).intersection(required_skills)
    missing_skills = required_skills - matched_skills

    if len(matched_skills) >= len(required_skills) * 0.9:
        suitability = "Suitable"
        advice = "✅ Your resume matches the job requirements."
    else:
        suitability = "Unsuitable"
        advice = f"⚠️ You are missing some key skills: {', '.join(missing_skills)}"

    return suitability, advice

# Flask Route
@app.route('/', methods=['GET', 'POST'])
def index():
    prediction, suitability, advice = "", "", ""

    if request.method == 'POST':
        file = request.files['resume']
        if not file.filename.endswith(('.pdf', '.docx')):
            return "⚠️ Unsupported File Type. Please upload a PDF or DOCX file."

        file_path = "uploaded_resume.docx"
        file.save(file_path)
        text = extract_text_from(file_path)

        job_field = request.form['job_field']
        prediction = predict_job_category(text)  # still show ML prediction
        suitability, advice = check_suitability(job_field, extracted_skills)

        #prediction = predict_job_category(text)
        #extracted_skills = extract_skills(text)
        #suitability, advice = check_suitability(prediction, extracted_skills)

    return render_template('index.html', prediction=prediction, suitability=suitability, advice=advice)

if __name__ == '__main__':
    app.run(debug=True)
